In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.linear_model import SGDRegressor


-9223372036854775808

In [2]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt 
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [3]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [4]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

## PREPROCESSING

In [5]:
df_train.columns[df_train.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [6]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [7]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [8]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [9]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [10]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [11]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [12]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [13]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [14]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [15]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [16]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [17]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [18]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int" or df_train[cols].dtype=="float":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
            elif for_numerical_cols=="mode":
                mode_=df_train[cols].mode()[0]
                df_train[cols].fillna(mode_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [19]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [20]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [21]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [22]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

Here we are encoding our categorical columns using 'one_hot_encoding' as a strategy. We will be experimenting other strategies later.

In [23]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [24]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [25]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,0.0,978,0,284,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1970,0.0,216,0,540,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,2000,0.0,0,0,953,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6,1988,119.0,790,163,589,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,9,2006,0.0,275,0,877,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6,1996,0.0,49,1029,0,...,0,0,0,1,0,0,0,0,1,0


In [26]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,69.863699,10516.828082,6.099315,5.575342,1984.865753,103.117123,443.639726,46.549315,567.240411,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,22.027677,9981.264932,1.382997,1.112799,20.645407,180.731373,456.098091,161.319273,441.866955,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,60.000000,7553.500000,5.000000,5.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,79.000000,11601.500000,7.000000,6.000000,2004.000000,164.250000,712.250000,0.000000,808.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


Taking features in X and lables in Y

In [27]:
X=df_train
Y=output_col

The below dictionary will save the final accuracies of each model.

-inf

In [42]:
avg_accuracies={}
highest_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    avg_accuracies[al]=0
    highest_accuracies[al]=float('-inf')

In [43]:
def algo_testing(algo:str,test_size:float,X,Y,evaluation_metric:str,random_state,shuffle):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=random_state,shuffle=shuffle)
   
    model.fit(X_train,Y_train)
    predicted=model.predict(X_test)
    
    r2=evaluation_metric(Y_test, predicted)
    print(f"Testing r2_score of {model} is ",r2)
    print(f"Training r2_score of {model} is ", model.score(X_train,Y_train))
    if highest_accuracies[algo]<r2:
        
        highest_accuracies[algo]=r2
    avg_accuracies[algo]+=r2


In [44]:
for i in range(50):
    algo_testing(algo="LinearRegression()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["LinearRegression()"]/=50

Testing r2_score of LinearRegression() is  0.8935049785069415
Training r2_score of LinearRegression() is  0.9293533416919844
Testing r2_score of LinearRegression() is  0.8212196078003644
Training r2_score of LinearRegression() is  0.9403486372406431
Testing r2_score of LinearRegression() is  0.8998114219075111
Training r2_score of LinearRegression() is  0.9300077904749179
Testing r2_score of LinearRegression() is  0.862841574705971
Training r2_score of LinearRegression() is  0.9296260546739724
Testing r2_score of LinearRegression() is  0.8924103057481335
Training r2_score of LinearRegression() is  0.9329069974656103
Testing r2_score of LinearRegression() is  0.8163546128107704
Training r2_score of LinearRegression() is  0.9435678571313513
Testing r2_score of LinearRegression() is  0.6337216243021624
Training r2_score of LinearRegression() is  0.9351541657787547
Testing r2_score of LinearRegression() is  0.806061247156181
Training r2_score of LinearRegression() is  0.9446552273605358
Te

In [45]:
for i in range(50):

    algo_testing(algo="RandomForestRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["RandomForestRegressor()"]/=50

Testing r2_score of RandomForestRegressor() is  0.8662311443749418
Training r2_score of RandomForestRegressor() is  0.9787236284584496
Testing r2_score of RandomForestRegressor() is  0.8457643918897612
Training r2_score of RandomForestRegressor() is  0.9769957403860364
Testing r2_score of RandomForestRegressor() is  0.8974859028511274
Training r2_score of RandomForestRegressor() is  0.9788648930836232
Testing r2_score of RandomForestRegressor() is  0.8510993025150816
Training r2_score of RandomForestRegressor() is  0.9770099610574068
Testing r2_score of RandomForestRegressor() is  0.8320710349268102
Training r2_score of RandomForestRegressor() is  0.977372578273134
Testing r2_score of RandomForestRegressor() is  0.9020052502250128
Training r2_score of RandomForestRegressor() is  0.9770152093429115
Testing r2_score of RandomForestRegressor() is  0.8662907796482648
Training r2_score of RandomForestRegressor() is  0.9809090126196554
Testing r2_score of RandomForestRegressor() is  0.886314

In [46]:
for i in range(50):

    algo_testing(algo="SVR()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SVR()"]/=50

Testing r2_score of SVR() is  -0.06887557336691996
Training r2_score of SVR() is  -0.05347213569993525
Testing r2_score of SVR() is  -0.10567597179265076
Training r2_score of SVR() is  -0.04972933457262396
Testing r2_score of SVR() is  -0.0581814211274041
Training r2_score of SVR() is  -0.04550754661163481
Testing r2_score of SVR() is  -0.06321230798888178
Training r2_score of SVR() is  -0.04742606614351619
Testing r2_score of SVR() is  -0.018371730194582714
Training r2_score of SVR() is  -0.052314727805618855
Testing r2_score of SVR() is  -0.07831141791385487
Training r2_score of SVR() is  -0.05087806770952641
Testing r2_score of SVR() is  -0.048829081669532926
Training r2_score of SVR() is  -0.05952382476185347
Testing r2_score of SVR() is  -0.06997824475334324
Training r2_score of SVR() is  -0.04823903812622299
Testing r2_score of SVR() is  -0.08859820851302214
Training r2_score of SVR() is  -0.052267340966273945
Testing r2_score of SVR() is  -0.04503382558863067
Training r2_score o

In [47]:
for i in range(50):

    algo_testing(algo="KNeighborsRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["KNeighborsRegressor()"]/=50

Testing r2_score of KNeighborsRegressor() is  0.5718886946211296
Training r2_score of KNeighborsRegressor() is  0.7690698549513137
Testing r2_score of KNeighborsRegressor() is  0.5383238873139963
Training r2_score of KNeighborsRegressor() is  0.7723641518999639
Testing r2_score of KNeighborsRegressor() is  0.6366487792967264
Training r2_score of KNeighborsRegressor() is  0.7484706946646675
Testing r2_score of KNeighborsRegressor() is  0.541454479569272
Training r2_score of KNeighborsRegressor() is  0.7538100908381189
Testing r2_score of KNeighborsRegressor() is  0.6564703606066038
Training r2_score of KNeighborsRegressor() is  0.7453209104564292
Testing r2_score of KNeighborsRegressor() is  0.6476635252524587
Training r2_score of KNeighborsRegressor() is  0.7476295257440676
Testing r2_score of KNeighborsRegressor() is  0.6584804221151335
Training r2_score of KNeighborsRegressor() is  0.7485055468734185
Testing r2_score of KNeighborsRegressor() is  0.6123306382144562
Training r2_score o

In [48]:
for i in range(50):

    algo_testing(algo="SGDRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SGDRegressor()"]/=50

Testing r2_score of SGDRegressor() is  -1.2372702199248037e+23
Training r2_score of SGDRegressor() is  -1.1816330478743512e+23
Testing r2_score of SGDRegressor() is  -1.1477432945648987e+24
Training r2_score of SGDRegressor() is  -1.2475473154643787e+24
Testing r2_score of SGDRegressor() is  -1.1965415046796715e+23
Training r2_score of SGDRegressor() is  -1.7224966114984083e+23
Testing r2_score of SGDRegressor() is  -3.538538511373584e+22
Training r2_score of SGDRegressor() is  -7.192377808329319e+22
Testing r2_score of SGDRegressor() is  -2.2655143093761634e+24
Training r2_score of SGDRegressor() is  -1.796611644343738e+24
Testing r2_score of SGDRegressor() is  -1.433197992585071e+24
Training r2_score of SGDRegressor() is  -3.252953823447107e+24
Testing r2_score of SGDRegressor() is  -4.313740433809387e+23
Training r2_score of SGDRegressor() is  -4.304969711778289e+23
Testing r2_score of SGDRegressor() is  -6.492313502060413e+23
Training r2_score of SGDRegressor() is  -6.2545286390797

In [49]:
for i in range(50):

    algo_testing(algo="DecisionTreeRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["DecisionTreeRegressor()"]/=50

Testing r2_score of DecisionTreeRegressor() is  0.7026428331091032
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6198393739408727
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6111335791378106
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6022121298890429
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.528411196055711
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6976949827333224
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.65740000396544
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.5667432239208714
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor()

In [50]:
for i in range(50):

    algo_testing(algo="ElasticNet()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["ElasticNet()"]/=50

Testing r2_score of ElasticNet() is  0.8568342104485038
Training r2_score of ElasticNet() is  0.8181370604450846
Testing r2_score of ElasticNet() is  0.8075728621167558
Training r2_score of ElasticNet() is  0.826032913033324
Testing r2_score of ElasticNet() is  0.8804110774235594
Training r2_score of ElasticNet() is  0.8165029154452121
Testing r2_score of ElasticNet() is  0.5640562345557849
Training r2_score of ElasticNet() is  0.8674911741032764
Testing r2_score of ElasticNet() is  0.4107452317664054
Training r2_score of ElasticNet() is  0.8863000064166785
Testing r2_score of ElasticNet() is  0.7814122384309314
Training r2_score of ElasticNet() is  0.8378709111538497
Testing r2_score of ElasticNet() is  0.868079596178403
Training r2_score of ElasticNet() is  0.8226342058479287
Testing r2_score of ElasticNet() is  0.5045006930739966
Training r2_score of ElasticNet() is  0.8619490317702063
Testing r2_score of ElasticNet() is  0.810015096042066
Training r2_score of ElasticNet() is  0.830

In [51]:
for i in range(50):

    algo_testing(algo="Lasso()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Lasso()"]/=50

C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.623e+11, tolerance: 7.305e+08
  model = cd_fast.enet_coordinate_descent(


Testing r2_score of Lasso() is  0.9124542316107445
Training r2_score of Lasso() is  0.9287190649608058


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.069e+10, tolerance: 7.291e+08
  model = cd_fast.enet_coordinate_descent(


Testing r2_score of Lasso() is  0.8195490377342081
Training r2_score of Lasso() is  0.9415148468873313


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.950e+11, tolerance: 7.214e+08
  model = cd_fast.enet_coordinate_descent(


...

C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.269e+11, tolerance: 7.114e+08
  model = cd_fast.enet_coordinate_descent(


Testing r2_score of Lasso() is  0.8876697244842211
Training r2_score of Lasso() is  0.9327531336674267


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.436e+11, tolerance: 7.204e+08
  model = cd_fast.enet_coordinate_descent(


Testing r2_score of Lasso() is  0.8882877862308777
Training r2_score of Lasso() is  0.9327869202914201
Testing r2_score of Lasso() is  0.5729849949418722
Training r2_score of Lasso() is  0.9404780445997689


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.045e+10, tolerance: 7.239e+08
  model = cd_fast.enet_coordinate_descent(


In [52]:
for i in range(50):

    algo_testing(algo="Ridge()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Ridge()"]/=50

Testing r2_score of Ridge() is  0.8166345655976405
Training r2_score of Ridge() is  0.9250332084716424
Testing r2_score of Ridge() is  0.6371441790452618
Training r2_score of Ridge() is  0.9258646160969615
Testing r2_score of Ridge() is  0.8830309658477365
Training r2_score of Ridge() is  0.9182135216443802
Testing r2_score of Ridge() is  0.8389668626107586
Training r2_score of Ridge() is  0.9228708631898683
Testing r2_score of Ridge() is  0.8796063548806753
Training r2_score of Ridge() is  0.9177829215820916
Testing r2_score of Ridge() is  0.8326427988688343
Training r2_score of Ridge() is  0.9266980476471457
Testing r2_score of Ridge() is  0.8116971421980159
Training r2_score of Ridge() is  0.9250136027205549
Testing r2_score of Ridge() is  0.6666671885467681
Training r2_score of Ridge() is  0.9315013335674058
Testing r2_score of Ridge() is  0.8941490638491663
Training r2_score of Ridge() is  0.9176703297880209
Testing r2_score of Ridge() is  0.6367392227694755
Training r2_score of R

In [53]:
avg_accuracies

{'LinearRegression()': 0.8277739642181668,
 'RandomForestRegressor()': 0.8446928289516914,
 'SVR()': -0.05572982516783045,
 'KNeighborsRegressor()': 0.6119395282311413,
 'SGDRegressor()': -1.4011753188280607e+24,
 'DecisionTreeRegressor()': 0.665957981400488,
 'ElasticNet()': 0.7564323675671808,
 'Lasso()': 0.7858295263752612,
 'Ridge()': 0.7915380283519825}

In [54]:
highest_accuracies

{'LinearRegression()': 0.91185128500907,
 'RandomForestRegressor()': 0.9020052502250128,
 'SVR()': -0.010529382533857934,
 'KNeighborsRegressor()': 0.7164166515488228,
 'SGDRegressor()': -2.5520904127831943e+22,
 'DecisionTreeRegressor()': 0.8378726958264279,
 'ElasticNet()': 0.905837886149564,
 'Lasso()': 0.9150432406213429,
 'Ridge()': 0.9078105110219898}